In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname("__file__"), "../..")) # directory of module fea_util
from contextlib import contextmanager

import msprime, pyslim
import numpy as np

from feature_util import run_RELATE

In [2]:
#parent_dir = os.getcwd() # directory where the job is submitted
args = ["slimSims2ts.py",
        "/Volumes/mo/maiz/maize_scale_40.param",
        40,
        's',
        '/Volumes/mo/maiz/SLiM_trial_swp_meta.txt',
        14,
        50,
        "SLiM_trial_swp",
        "SLiM_trial_swp"]

param_path = args[1]
scale = int(args[2])
mode = args[3]
if mode == 'n':
    no_sims = int(args[4])
elif mode == 's':
    metaF = args[4]
# else:
#     return helpMsg
thr = int(args[5]) # should be 1-indexed
tot_thr = int(args[6])
inPref = args[7]
outPref = args[8]

In [3]:
with open(param_path, "r") as paramF:
    lines = paramF.readlines()
scaled_mu = float(lines[0].strip())
scaled_rho = float(lines[1].strip())
scaled_N0 = int(lines[-1].strip().split()[1])

mu = scaled_mu/scale
rho_cMpMb = scaled_rho/scale*100*1e6 # 1cM = 1e-2 crossover
N0 = scaled_N0*scale

In [4]:
print(scaled_mu, scaled_rho, scaled_N0)
print(mu, rho_cMpMb, N0)

1.2e-06 2.79e-07 2501435
3e-08 0.6975 100057400


In [5]:
if mode == 's':
    meta_data = np.genfromtxt(metaF, usecols=(1, 2, 3, 4), dtype=None)
    # e.g. `%% 0.100992 2347 0.520421 SLiM_trial_swp/SLiM_trial_swp_4501`
    no_sims = meta_data.shape[0]

/Users/mo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


In [6]:
meta_data.shape

(45939,)

In [7]:
#thr = 14
tasks = no_sims//tot_thr
a_idx = (thr-1)*tasks # inclusive
if thr == tot_thr:
    b_idx = no_sims # exclusive
else:
    b_idx = thr*tasks # exclusive

print("Processing: [", a_idx, b_idx, ")", flush=True)

Processing: [ 11934 12852 )


In [ ]:
wd = outPref+'/'+'RELATE_temp_'+str(thr)
os.mkdir(wd, 0o755)

In [8]:
r_idx = 11934
parent_dir = "/Volumes/mo/maiz"

In [9]:
ID = int(meta_data[r_idx][3].split(b'_')[-1]) # retrieve 1-based index from meta file
sim_path = parent_dir+"/"+meta_data[r_idx][3].decode()+"_samp.trees"

In [10]:
print(ID, sim_path)

15459 /Volumes/mo/maiz/SLiM_trial_swp/SLiM_trial_swp_15459_samp.trees


In [11]:
os.path.isfile(sim_path)

True

In [12]:
def get_site_ppos(ts):
    var_ppos_ls = []
    prev_pos = 0
    for site in ts.sites():
        site_pos = int(site.position)
        if site_pos <= prev_pos:
            if prev_pos == 49999:
                var_ppos_ls.append(-1) # flag indicating this site should be removed
                continue
            else:
                site_pos = prev_pos + 1
        var_ppos_ls.append(site_pos)
        prev_pos = site_pos
    return np.array(var_ppos_ls)

def sim2ts(ts_mut, mu, rho_cMpMb, N0):

    # ts_samp = pyslim.load(sim_path)
    # ts_mut = msprime.mutate(ts_samp, rate=scaled_mu, keep=True) # random_seed=958,

    ppos_ls = get_site_ppos(ts_mut)
    GTM = ts_mut.genotype_matrix()
    
    mask = (ppos_ls != -1)
    ppos_ls = ppos_ls[mask]
    GTM = GTM[mask, :]

    ts_inf, _ = run_RELATE(ppos_ls, GTM, str(2*N0), rho_cMpMb=rho_cMpMb, mut_rate=str(mu))

    return ts_inf # inferred tree-seqs

In [13]:
print("Input:", sim_path, flush=True)
ts_samp = pyslim.load(sim_path)
ts_tru = msprime.mutate(ts_samp, rate=scaled_mu, keep=True)
ts_inf = sim2ts(ts_tru, mu, rho_cMpMb, N0)

Input: /Volumes/mo/maiz/SLiM_trial_swp/SLiM_trial_swp_15459_samp.trees
Milestone: Running RELATE pipeline, try_1


FileNotFoundError: [Errno 2] No such file or directory: '/sonas-hs/siepel/hpc_norepl/home/mo/relate_v1.0.17_x86_64_static/bin/Relate': '/sonas-hs/siepel/hpc_norepl/home/mo/relate_v1.0.17_x86_64_static/bin/Relate'

In [14]:
print(parent_dir+"/"+outPref+"/"+outPref+"_"+str(ID)+"_inf.trees")

/Volumes/mo/maiz/SLiM_trial_swp/SLiM_trial_swp_15459_inf.trees


In [15]:
print(ID)
print(meta_data[r_idx][0])
print(meta_data[r_idx][1])
print(meta_data[r_idx][2])

15459
0.817875
2478
0.255279


In [18]:
ts_tru.dump("test_tru.trees")